In [ ]:
%cd ..

In [ ]:
import sys

sys.path.append('src')

In [ ]:
from importlib import reload
from multiprocessing import Pool
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

import numpy as np
from tqdm.auto import tqdm

import utils
import data
import predictors
import metrics

In [ ]:
reload(data)
reload(utils)
reload(predictors)

In [ ]:
%%time
vocabulary = utils.Vocabulary.load(Path('vocabulary.csv'))
grids = utils.load_grids('grids.json')
# candidate_generator = predictors.FirstLetterCandidateGenerator(vocabulary, grids)
candidate_generator = predictors.FirstLetterLengthCandidateGenerator(vocabulary, grids)
ranker = predictors.InterpolatedDTWRanker(100.)
predictor = predictors.Predictor(candidate_generator, ranker)

In [ ]:
reader = data.TracesReader('data/valid_500.txt')
x = list(reader)
y = data.read_words(Path('data/valid_500.ref'))

In [ ]:
reader = data.TracesReader('data/test.txt')
x = list(reader)

In [ ]:
%%time
with ProcessPoolExecutor(8) as executor:
    res = list(tqdm(executor.map(predictor, x), total=len(x)))
    # list((pool.map(predictor, (reader))))

In [ ]:
%%time
list(tqdm(map(predictor, x), total=len(x)))

In [ ]:
%%time
with Pool(8) as executor:
    res = list(tqdm(executor.map(predictor, x), total=len(x)))
    # list((pool.map(predictor, (reader))))

In [ ]:
metrics.mrr_iterables(res, y)

In [ ]:
data.save_results('result/test_interp_100.csv', res)